In [1]:
import numpy as np
import pandas as pd

In [2]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

sample_submission.csv
test.csv
train.csv



In [4]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
max_features = 20000
maxlen = 100

In [6]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
train = train.sample(frac=1)

In [7]:
train.head()

id                                       comment_text  \
154640  b0f437119b56c6f4  "\nSo I'm still waiting for an actual reason. ...   
153091  972f3e1770dcd676  "\n\nNOTE\n and  continue to vandalize this pa...   
136092  d7dfd06388dafe24                             hey buddy \n\nFUCK YOU   
146372  2b2abfa22ba63b2f  ":::Even during the 1891 census, castes like V...   
106996  3c1aad0bb324236f            And...\nTELL THE WIKITRUTH.75.21.101.63   

        toxic  severe_toxic  obscene  threat  insult  identity_hate  
154640      0             0        0       0       0              0  
153091      0             0        0       0       0              0  
136092      1             1        1       0       1              0  
146372      0             0        0       0       0              0  
106996      0             0        0       0       0              0

In [9]:
list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [10]:
list_sentences_train

array(['"\nSo I\'m still waiting for an actual reason.  Provide a source that disputes him as the GOAT.  Better yet, provide several so that you can say that he is NOT ""widely considered the GOAT"".  As opposed to the fact that he IS ""widely considered the GOAT"" because there are myriads of sources that support the latter statement. "',
       '"\n\nNOTE\n and  continue to vandalize this page by including non-sourced information pertaining to a boyfriend named ""Chad Eagleton.""  TXmaster uploaded a picture on 2/17 in which he states that he is Chad Eagleton.  These claims are most likely dubious attempts by the two users (who are probably actually the same person) to claim that they are dating Carrie Underwood.  As none of their claims are sourced, I along with several other users continue to delete them.  Neither user has stepped up to the challenge of providing a source for their claims.  Please continue to delete their constant vandalism.  "',
       'hey buddy \n\nFUCK YOU', ..

In [12]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [13]:
list_sentences_test

array(["Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,",
       '== From RfC == \n\n The title is fine as it is, IMO.',
       '" \n\n == Sources == \n\n * Zawe Ashton on Lapland —  /  "', ...,
       '" \n\n == Okinotorishima categories == \n\n I see your changes and agree this is ""more correct.""  I had gotten confused, but then found this: \n :... while acknowledging Japan\'s territorial rights to Okinotorishima itself ... \n However, is there a category for  \n :... did not acknowledge Japan\'s claim to an exclusive economic zone (EEZ) stemming from Okinotorishima. \n That is, is there a category for ""disputed EEZ""s?   "',
       '" \n\n == ""One of the founding n

In [15]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [16]:
X_t

array([[   0,    0,    0, ...,    1, 1873,  446],
       [   0,    0,    0, ...,   92, 2728,  199],
       [   0,    0,    0, ..., 3583,  129,    6],
       ...,
       [   0,    0,    0, ...,   10,    1,   23],
       [   0,    0,    0, ...,    2,  519, 1416],
       [1556,   32,   57, ..., 2280,  469,  182]], dtype=int32)

In [17]:
X_te

array([[   0,    0,    0, ...,  145,  493,   84],
       [   0,    0,    0, ...,   11,    8, 2828],
       [   0,    0,    0, ...,  109,   15,  355],
       ...,
       [   0,    0,    0, ...,   12, 1652,  358],
       [   0,    0,    0, ..., 9857, 3507,  355],
       [   0,    0,    0, ...,  100, 5216,    6]], dtype=int32)

In [18]:
def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
                 )
    
    return model

In [19]:
model = get_model()
batch_size = 32
epochs = 2

In [20]:
file_path = "weights_base.best.hdf5"

In [21]:
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose = 1, save_best_only=True, mode='min')

In [23]:
checkpoint

In [22]:
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

In [24]:
early

In [26]:
callbacks_list = [checkpoint, early]
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 1640s 11ms/step - loss: 0.0643 - acc: 0.9788 - val_loss: 0.0502 - val_acc: 0.9821

Epoch 00001: val_loss improved from inf to 0.05020, saving model to weights_base.best.hdf5
Epoch 2/2
143613/143613 [==============================] - 1631s 11ms/step - loss: 0.0459 - acc: 0.9831 - val_loss: 0.0467 - val_acc: 0.9829

Epoch 00002: val_loss improved from 0.05020 to 0.04668, saving model to weights_base.best.hdf5


In [27]:
model.load_weights(file_path)

In [28]:
y_test = model.predict(X_te)

In [29]:
sample_submission = pd.read_csv("../input/sample_submission.csv")

In [30]:
sample_submission[list_classes] = y_test

In [31]:
sample_submission.to_csv("../submission/submission_03.csv", index=False)